In [1]:
from pyspark.sql import SparkSession
import findspark

findspark.init()
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()


In [3]:
import os
import pymongo
import pandas as pd
from dotenv import load_dotenv
load_dotenv('../../config/.env')
mongo_uri = os.getenv('MONGO_URI')
client = pymongo.MongoClient(f'{mongo_uri}')
db = client['name']
collection = db['thai_name']


document = list(collection.find({},{ "_id": 0,}))
client.close()



ฺื**NOT USED**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

df = pd.DataFrame(document)
df['gender'] = df['gender'].map({'f': 0, 'm': 1})

cv = CountVectorizer()
X = cv.fit_transform(df['name'].values.astype('U'))
cv.get_feature_names_out()

y = df['gender']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train, y_train)
nb.score(X_test, y_test)




,name,gender
0,Abduloh,m
1,Abdulrohman,m
2,Abdulroman,m
3,Abhisit,m
4,Achara,f


0.5011990407673861

Use this one, added Feature on Naive Bayes, will add pipeline and pickle later

In [108]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier

def features(name):
    name = str(name)
    name = name.lower()
    return {
        'first-letter': name[0],
        'last-letter': name[-1],
        'first2-letter': name[:2],
        'last2-letter': name[-2:],
        'first3-letter': name[:3],
        'last3-letter': name[-3:],
        
    }
    
features = np.vectorize(features)

df_X = features(df['name'])
df_y = df['gender']



# dv.fit(names_sample)
# transformed = dv.transform(names_sample)
# transformed.toarray()

# dv.get_feature_names_out()

X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.3, random_state=2024)

dv = DictVectorizer()
dv.fit_transform(X_train)


rf = RandomForestClassifier()
rf.fit(dv.transform(X_train), y_train)
rf.score(dv.transform(X_test), y_test)

report = classification_report(y_test, rf.predict(dv.transform(X_test)))

In [109]:
sample = ['Nithi', 'Rawit', 'Thamvarut', 'Pasit', 'Peeradon', 'Kittiphop', 'Nigga', 'Aranya', 'Kamon', 'Dao',]
transform = dv.transform(features(sample)).toarray()
rf.predict(transform)


array(['m', 'm', 'm', 'm', 'm', 'm', 'f', 'f', 'm', 'm'], dtype=object)

for pickle

In [130]:
import pickle
from sklearn.pipeline import FunctionTransformer, Pipeline


df = pd.DataFrame(document)
df = df[~df['name'].duplicated(keep=False)]

print(df['name'].value_counts().sort_values())

def features(name):
    name = str(name).lower()
    return {
        'first-letter': name[0],
        'last-letter': name[-1],
        'first2-letter': name[:2],
        'last2-letter': name[-2:],
        'first3-letter': name[:3],
        'last3-letter': name[-3:]
    }
    
X = df['name']
y = df['gender']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

pipeline = Pipeline([
    ('feature_extractor', FunctionTransformer(np.vectorize(features), validate=False)),
    ('vectorizer', DictVectorizer()),
    ('classifier', MultinomialNB())
])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

with open('thai_name_pipeline.pkl', 'wb') as f:
    pickle.dump(pipeline, f)

name
Abduloh       1
Adsada        1
Abdulroman    1
Abhisit       1
Achara        1
             ..
Yutthana      1
Yutthasak     1
Yutthasat     1
Yuwadee       1
Yuwadi        1
Name: count, Length: 3151, dtype: int64
              precision    recall  f1-score   support

           f       0.82      0.82      0.82       240
           m       0.89      0.89      0.89       391

    accuracy                           0.87       631
   macro avg       0.86      0.86      0.86       631
weighted avg       0.87      0.87      0.87       631



In [135]:
with open('thai_name_pipeline.pkl', 'rb') as f:
    load_pipe = pickle.load(f)
    
names = ['Nithi', 'Rawit', 'Thamvarut', 'Pasit', 'Peeradon', 'Kittiphop', 'Nigga', 'Aranya', 'Kamon', 'Dao',]

report = load_pipe.predict(names)
print(report)

['m' 'm' 'm' 'm' 'm' 'm' 'f' 'f' 'f' 'f']
